# Lab Five: Wide and Deep Networks

Team: Miro Ronac, Kirk Watson, Brandon Vincitore

## 1. Preparation

Dataset Source: https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset


### 1.1
TODO Define and prepare your class variables. 

TODO Use proper variable representations (int, float, one-hot, etc.).

TODO Use pre-processing methods (as needed) for dimensionality reduction, scaling, etc.

TODO Remove variables that are not needed/useful for the analysis.

TODO Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler

df = pd.read_csv('diabetes_012_health_indicators_BRFSS2015.csv')
print('Initial size of dataset:', df.shape[0])

#drop instances with NA values if any
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)
print('Size of dataset after removing NA values:', df.shape[0])


#categorical data
#Diabetes_012, but its the prediction task
#GenHlth - Would you say that in general your health is: scale 1-5 1 = excellent 2 = very good 3 = good 4 = fair 5 = poor
#Age - 13-level age category (_AGEG5YR see codebook) 1 = 18-24 9 = 60-64 13 = 80 or older
#Education - Education level (EDUCA see codebook) scale 1-6 1 = Never attended school or only kindergarten 2 = Grades 1 through 8 (Elementary) 3 = Grades 9 through 11 (Some high school) 4 = Grade 12 or GED (High school graduate) 5 = College 1 year to 3 years (Some college or technical school) 6 = College 4 years or more (College graduate)
#Income - Income scale (INCOME2 see codebook) scale 1-8 1 = less than $10,000 5 = less than $35,000 8 = $75,000 or more

#continuous
#BMI
#MentHlth - or large categorical? max 30 days
#PhysHlth - or large categorical? max 30 days

#normalize continuous data
continuous = ['BMI','MentHlth','PhysHlth']
std_scaler = StandardScaler()
df[continuous] = std_scaler.fit_transform(df[continuous])
df.head()



all features are relevant and don't need to be removed

all categorical data is already represented as integers, no need to encode



In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder, StandardScaler

df = pd.read_csv('diabetes_012_health_indicators_BRFSS2015.csv')

# Making target feature binary
target_array = np.zeros((len(df),4))
for i in range(len(df)):
    target_array[i,0] = df['Diabetes_012'].values[i]
for i in range(len(target_array)):
    # no diabetes
    if target_array[i,0] == 0:
        target_array[i,1] = 1
    # prediabetes
    if target_array[i,0] == 1:
        target_array[i,2] = 1
    # diabetes
    if target_array[i,0] == 2:
        target_array[i,3] = 1

# Adding new target columns to original dataframe
target_headers = ['NoDiabetes', 'PreDiabetes', 'Diabetes']
for i in range(target_array.shape[1]-1):
    df.insert(i, target_headers[i], target_array[:,1:][:,i], True)
df_target = df.drop('Diabetes_012', axis=1) # removing previous categorical target column
df_target.drop(df_target.tail(252000).index,inplace = True)
    
print(df_target.info())

#show df
pd.set_option('display.max_columns', None)
df_target

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   NoDiabetes            1680 non-null   float64
 1   PreDiabetes           1680 non-null   float64
 2   Diabetes              1680 non-null   float64
 3   HighBP                1680 non-null   float64
 4   HighChol              1680 non-null   float64
 5   CholCheck             1680 non-null   float64
 6   BMI                   1680 non-null   float64
 7   Smoker                1680 non-null   float64
 8   Stroke                1680 non-null   float64
 9   HeartDiseaseorAttack  1680 non-null   float64
 10  PhysActivity          1680 non-null   float64
 11  Fruits                1680 non-null   float64
 12  Veggies               1680 non-null   float64
 13  HvyAlcoholConsump     1680 non-null   float64
 14  AnyHealthcare         1680 non-null   float64
 15  NoDocbcCost          

,NoDiabetes,PreDiabetes,Diabetes,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1.0,0.0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,1.0,0.0,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,1.0,0.0,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,1.0,0.0,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,1.0,0.0,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1675,1.0,0.0,0.0,1.0,0.0,1.0,20.0,0.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,13.0,5.0,4.0
1676,0.0,0.0,1.0,0.0,1.0,1.0,30.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,4.0,0.0,25.0,1.0,0.0,9.0,5.0,3.0
1677,1.0,0.0,0.0,0.0,0.0,1.0,42.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,2.0,15.0,0.0,0.0,0.0,8.0,6.0,8.0
1678,0.0,0.0,1.0,1.0,1.0,1.0,28.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,4.0,15.0,22.0,1.0,1.0,11.0,6.0,5.0


In [6]:
# Check for missing values
df_target.isnull().sum().any()

False

In [7]:
# Encoding categorical features and normalizing numerical variables
from sklearn.preprocessing import LabelEncoder 
from sklearn.preprocessing import StandardScaler
# ========================================================
from copy import deepcopy
df_encode = deepcopy(df_target)
# ========================================================
# for every categorical variable, encode as integer
# Keras will use the integer variable to figure out how to one-hot encode    
encoders = dict() # save each encoder in dictionary
categorical_headers = ['GenHlth','Age','Education','Income']

for col in categorical_headers:
        encoders[col] = LabelEncoder() # save the encoder
        df_encode[col] = encoders[col].fit_transform(df_encode[col])
# ========================================================
# scale the numeric, continuous variables
numerical_headers = ["BMI", "MentHlth", "PhysHlth"]

for col in numerical_headers:
    ss = StandardScaler()
    df_encode[col] = ss.fit_transform(df_encode[col].values.reshape(-1,1))
# ========================================================
# making binary variables integers
binary_headers = ['NoDiabetes','PreDiabetes','Diabetes','HighBP','HighChol','CholCheck','Smoker','Stroke','HeartDiseaseorAttack',
                  'PhysActivity','Fruits','Veggies','HvyAlcoholConsump','AnyHealthcare','NoDocbcCost','DiffWalk','Sex',]
for col in binary_headers:
    df_encode[col] = df_encode[col].astype(int)
df_encode.head()

,NoDiabetes,PreDiabetes,Diabetes,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,Veggies,HvyAlcoholConsump,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1,0,0,1,1,1,1.703564,1,0,0,0,0,1,0,1,0,4,1.727369,0.963016,1,0,8,3,2
1,1,0,0,0,0,0,-0.672844,1,0,0,1,0,0,0,0,1,2,-0.477318,-0.564619,0,0,6,5,0
2,1,0,0,1,1,1,-0.197562,0,0,0,0,1,0,0,1,1,4,3.197160,2.490651,1,0,8,3,7
3,1,0,0,1,0,1,-0.355990,0,0,0,1,1,1,0,1,0,1,-0.477318,-0.564619,0,0,10,2,5
4,1,0,0,1,1,1,-0.831271,0,0,0,1,1,1,0,1,0,1,-0.109870,-0.564619,0,0,10,4,3


In [8]:
# ========================================================
# Define features and target
X = df_encode.drop(target_headers, axis=1)
y = df_encode[target_headers]
# ========================================================
X.info()
y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1680 entries, 0 to 1679
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   HighBP                1680 non-null   int32  
 1   HighChol              1680 non-null   int32  
 2   CholCheck             1680 non-null   int32  
 3   BMI                   1680 non-null   float64
 4   Smoker                1680 non-null   int32  
 5   Stroke                1680 non-null   int32  
 6   HeartDiseaseorAttack  1680 non-null   int32  
 7   PhysActivity          1680 non-null   int32  
 8   Fruits                1680 non-null   int32  
 9   Veggies               1680 non-null   int32  
 10  HvyAlcoholConsump     1680 non-null   int32  
 11  AnyHealthcare         1680 non-null   int32  
 12  NoDocbcCost           1680 non-null   int32  
 13  GenHlth               1680 non-null   int64  
 14  MentHlth              1680 non-null   float64
 15  PhysHlth             

## cross features

In [11]:
categorical_headers = ['GenHlth','Age','Education','Income']
for col in categorical_headers:
    vals = df[col].unique()
    print(col,'has', len(vals), 'unique values:')
    print(vals)

GenHlth has 5 unique values:
[5. 3. 2. 4. 1.]
Age has 13 unique values:
[ 9.  7. 11. 10.  8. 13.  4.  6.  2. 12.  5.  1.  3.]
Education has 6 unique values:
[4. 6. 3. 5. 2. 1.]
Income has 8 unique values:
[3. 1. 8. 6. 4. 7. 2. 5.]


In [17]:
cross_columns = [['Education', 'Income'],
                 ['Education','Income','Age'],
                 ['GenHlth','Education','Age'],
                 ['GenHlth', 'Income']
                ]

for cols_list in cross_columns:
    enc = LabelEncoder()
    
    # 1. create crossed labels by join operation
    print(df_encode[cols_list])

      Education  Income
0             3       2
1             5       0
2             3       7
3             2       5
4             4       3
...         ...     ...
1675          4       3
1676          4       2
1677          5       7
1678          5       4
1679          3       5

[1680 rows x 2 columns]
      Education  Income  Age
0             3       2    8
1             5       0    6
2             3       7    8
3             2       5   10
4             4       3   10
...         ...     ...  ...
1675          4       3   12
1676          4       2    8
1677          5       7    7
1678          5       4   10
1679          3       5    8

[1680 rows x 3 columns]
      GenHlth  Education  Age
0           4          3    8
1           2          5    6
2           4          3    8
3           1          2   10
4           1          4   10
...       ...        ...  ...
1675        0          4   12
1676        3          4    8
1677        1          5    7
1678        3 

In [16]:
cross_columns = [['Education', 'Income'],
                 ['Education','Income','Age'],
                 ['GenHlth','Education','Age'],
                 ['GenHlth', 'Income']
                ]

cross_col_df_names= []
for cols_list in cross_columns:
    enc = LabelEncoder()
    
    # 1. create crossed labels by join operation
    X_crossed = df_encode[cols_list].apply(lambda x: '_'.join(x), axis=1)
    
    # get a nice name for this new crossed column
    cross_col_name = '_'.join(cols_list)
    # 2. encode as integers, stacking all possibilities
    enc.fit(X_crossed)
    
    # 3. Save into dataframe with new name
    df_encode[cross_col_name] = enc.transform(X_crossed)
    
    # keep track of the new names of the crossed columns
    cross_col_names.append(cross_col_name)
    
cross_col_df_names

TypeError: sequence item 0: expected str instance, int found

## test and training split

In [13]:
columns = list(df_target.columns)
for col in columns:
    df_target[col] = df_target[col].astype(int)

targets = ['NoDiabetes', 'PreDiabetes', 'Diabetes']
for col in targets:
    columns.remove(col)
    
# Splitting dataset
from sklearn.model_selection import train_test_split as tts

train, test = tts(df_target, test_size=.20, random_state=42, shuffle=True)

X_test  = test[columns].to_numpy()
X_train = train[columns].to_numpy()
y_test = test[targets].to_numpy()
y_train = train[targets].to_numpy()

## 2. Modeling

## 3. Exceptional Work